In [2]:
import pandas as pd

from database_read import read
from sqlalchemy import create_engine
import re
from datetime import datetime
order = pd.read_excel(r'E:/订单数量校准/订单数量校准2022-05-12-14_28_34.xlsx' )
notexist = pd.read_excel(r'E:/订单数量校准/订单不存在2022-05-12-14_28_34.xlsx' )
guonei_l = pd.read_excel(r'E:/国内补寄/国内补寄校对/2022-05-12-14_37_51补寄清单售后漏登记.xlsx' )
guowai_bufu =pd.read_excel(r'E:/登记补寄数量/国外补寄登记补寄数量不符/py2022-05-12-14_49_12登记补寄数量不符.xlsx' )
guowai_l = pd.read_excel(r'E:/登记补寄数量/国外补寄自发货订单售后漏登记/2022-05-12-12_16_07国外补寄自发货订单售后漏登记.xlsx' )
tuohuo_l =pd.read_excel(r'E:/绩效管理表/退货漏登/2022-05-11-17_29_01退货售后漏登记.xlsx' )
store = pd.read_excel(r'E:/订单数量校准/店铺错误/py店铺错误2022-05-12-14_33py.xlsx' )
# not_buji = pd.read_excel(r'E:/登记补寄数量/国外补寄登记补寄数量不符/py2022-05-12-18_52_14登记补寄数量不符.xlsx' )

order['原因'] = '订单登记条数不符合'
notexist['原因'] = '订单错填或不存在'
guonei_l['原因'] = '国内补寄漏登'
guowai_bufu['原因'] = '补寄数量不符'
guowai_l['原因'] = '国外补寄漏登'
tuohuo_l['原因'] = '退货漏登'
store['原因'] = '售后登记店铺不符'
not_buji['原因'] = '订单补寄数量不符'



In [3]:
order = order[['订单号','SKU','登记人','店铺','原因']]
notexist= notexist[['订单号','SKU','登记人','店铺','原因']]
guonei_l = guonei_l[['订单号','SKU',"寄件人","店铺",'原因']]
guonei_l = guonei_l.rename(columns={"寄件人":"登记人"})
guowai_bufu = guowai_bufu[['订单号','SKU','登记人','店铺','原因']] 
guowai_l = guowai_l[['平台订单号','账号','公司SKU',"运维",'原因'] ]
guowai_l= guowai_l.rename(columns = {"账号":"店铺","平台订单号":"订单号","公司SKU":"SKU","运维":"登记人"})
tuohuo_l = tuohuo_l[['订单号','公司SKU','负责人','店铺','原因'] ]
tuohuo_l= tuohuo_l.rename(columns = {"公司SKU":"SKU","负责人":"登记人"})
store= store[['订单号','SKU','登记人','店铺','库账号','原因']]
store = store.rename(columns = {"库账号":"自发货系统店铺"})
# not_buji = not_buji[['订单号','SKU','登记人','店铺','原因']]

In [4]:
print(len(order))
print(len(notexist))
print(len(guonei_l))
print(len(guowai_bufu))
print(len(guowai_l))
print(len(tuohuo_l))
# print(len(not_buji))

8
202
34
343
45
29
398


In [5]:
order.head(2)

,订单号,SKU,登记人,店铺,原因
0,111-9277740-7944228,USAN1026259-1,黄平,amazon驰甬,订单登记条数不符合
1,111-9277740-7944228,USAN1026259-1,黄平,amazon驰甬,订单登记条数不符合


In [6]:
notexist.head(2)

,订单号,SKU,登记人,店铺,原因
0,111-0371211-0953000,USAN1032096-9,童杰,amazoncpower,订单错填或不存在
1,111-0589264-0985836,USAN1026771-5,朱洁,amazon赫曼,订单错填或不存在


In [7]:
# guonei_l = guonei_l.rename(columns={'ID': 'id','创建时间': '订单时间'})
guonei_l.head(2)

,订单号,SKU,登记人,店铺,原因
0,112-3560313-8696234,USAN1023893-2,何晓敏,赫曼,国内补寄漏登
1,112-3560313-8696234,USAN1023893-2,何晓敏,赫曼,国内补寄漏登


In [8]:
guowai_bufu.head(2)

,订单号,SKU,登记人,店铺,原因
0,114-5002307-5673839,USAN1025837-1,何晓敏,amazon森月,补寄数量不符
1,111-8846098-9299450,USAN1022780-8,敖睿洁,amazon宫本,补寄数量不符


In [9]:
# guowai_l = guowai_l.rename(columns={'平台订单号': '订单号','账号': '店铺','公司SKU': 'SKU'})
guowai_l.head(2)

,订单号,店铺,SKU,登记人,原因
0,112-0969692-0156225,amazon森月,USAN1023801-1,夏银平,国外补寄漏登
1,112-9301468-7309025,amazon赫曼,USAN1023944-4,方婷,国外补寄漏登


In [10]:
# tuohuo_l = tuohuo_l.rename(columns={'负责人': '登记人','公司SKU': 'SKU'})
tuohuo_l.head(2)

,订单号,SKU,登记人,店铺,原因
0,112-6732398-2217060,USAN1022497-1,陈晓祥,赫曼,退货漏登
1,112-6141403-5733844,USAN1032096-11,胡程程,赫曼,退货漏登


In [11]:
store.head(2)

,订单号,SKU,登记人,店铺,自发货系统店铺,原因
0,111-7269941-9241027,USAN1029449-3,孙紫馨,amazon信盒,amazon维禄,售后登记店铺不符
1,112-6639455-7661825,USAN1026259-3,黄平,amazon信盒,amazon维禄,售后登记店铺不符


In [12]:
not_buji.head(2)

,订单号,SKU,登记人,店铺,原因
0,114-5002307-5673839,USAN1025837-1,何晓敏,amazon森月,订单补寄数量不符
1,111-8846098-9299450,USAN1022780-8,敖睿洁,amazon宫本,订单补寄数量不符


In [13]:
from functools import reduce
df = pd.DataFrame()
df_groups = [order,notexist,guowai_bufu,guonei_l,guowai_l,tuohuo_l,store,not_buji]
df = reduce(lambda left, right: pd.merge(left, right, on=['订单号','SKU','登记人','店铺','原因'], how = 'outer'), df_groups)
# df_groups1 = [df,guonei_l]
# df1 = reduce(lambda left, right: pd.merge(left, right, on=['订单号','SKU','登记人','店铺','原因'], how = 'outer'), df_groups1)
# df_groups2 = [df1,guowai_l]
# df2 = reduce(lambda left, right: pd.merge(left, right, on=['订单号','SKU','登记人','店铺','原因'], how = 'outer'), df_groups2)
# df_groups3 = [df2,tuohuo_l ]
# df3 = reduce(lambda left, right: pd.merge(left, right, on=['订单号','SKU','登记人','店铺','原因'], how = 'outer'), df_groups3)
df

,订单号,SKU,登记人,店铺,原因,自发货系统店铺
0,111-9277740-7944228,USAN1026259-1,黄平,amazon驰甬,订单登记条数不符合,NaN
1,111-9277740-7944228,USAN1026259-1,黄平,amazon驰甬,订单登记条数不符合,NaN
2,114-4895206-4949833,USAN1023801-3,夏银平,amazon森月,订单登记条数不符合,NaN
3,114-4895206-4949833,USAN1023801-3,夏银平,amazon森月,订单登记条数不符合,NaN
4,5863102420764,USAN1022780-8,李永芳,walmart赫曼,订单登记条数不符合,NaN
...,...,...,...,...,...,...
1159,111-8994981-4764240,USAN1035110-2,杨颖昕,amazon赫曼,订单补寄数量不符,NaN
1160,112-0925166-0489835,USAN1022780-1,杨颖昕,amazon赫曼,订单补寄数量不符,NaN
1161,114-4832250-6273813,USAN1035676-4,孙颖,amazon信盒,订单补寄数量不符,NaN
1162,114-2537497-6817804,USAN1023938-5,邓园园,amazon信盒,订单补寄数量不符,NaN


In [101]:
import time
now = time.strftime("%Y-%m-%d-%H_%M",time.localtime(time.time())) 
df = df.drop_duplicates()
df.to_excel(excel_writer="E:/"+now+r"整合.xlsx", index=False,encoding='utf-8')

In [13]:
import pandas as pd

from database_read import read
from sqlalchemy import create_engine
import re
from datetime import datetime
import time
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time())) 

qwe = pd.read_excel(r'E:/整合.xlsx' )
origin = read('after_sales','new_after_salesv2')
origin = origin.rename(columns={'订单号':'平台订单号','SKU':'公司SKU'})
qwe1 = pd.merge(qwe,origin,on =['平台订单号','公司SKU'])
qwe1.to_excel(excel_writer="E:/"+now+r"整合merge.xlsx", index=False,encoding='utf-8')


In [14]:
originzifa = pd.DataFrame()
for x in range(1,14):
    origin_zi = pd.read_table(r"E:/自发货订单管理源数据/系统数据/新数据_周/%s.txt" % x )
#     print(origin_zi[origin_zi['订单号'] == 'NF10834863947-4504431296678'])
    origin_zifa = pd.DataFrame(origin_zi, columns=['订单号', '账号', '平台SKU','公司SKU', '数量', '包裹号','订单类型', '订单状态', '创建时间'])
    originzifa = originzifa.append(origin_zifa)
    originzifa = originzifa.reset_index(drop=True)
originzifa = originzifa.rename(columns={'订单号': '平台订单号','创建时间': '订单时间'})
originzifa['订单时间'] =pd.to_datetime(originzifa['订单时间'])
# originzifa =originzifa[~originzifa["订单状态"].str.contains('已取消')]
originzifa.head(2)

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单类型,订单状态,订单时间
0,111-1984040-5269802,宫本,GB-103483-1,PTG103482-01,1,AK2020122500001,正常订单,已发货,2020-12-25 03:40:00
1,113-9678451-1602655,宫本,G-1015038-1,USAN1015038-1,1,AK2020122500005,正常订单,已发货,2020-12-25 05:57:00


In [64]:
originzifa[originzifa['平台订单号'] == '112-6201342-7128260']

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单类型,订单状态,订单时间
156613,112-6201342-7128260,优瑞斯特,U2-USAN1021245-3+4-a,NaN,1,NaN,正常订单,已取消,2021-05-17 12:37:00


In [56]:
originzifa[originzifa["订单状态"].str.contains('已取消')]['平台订单号'].unique()

array(['113-1420537-7250605', '111-1641081-4422669',
       '114-1427748-5893829', '113-9371577-6979437',
       '111-5127027-4278622', '113-6066225-6817015',
       '112-6731141-3069825', '111-2187766-6005813',
       '113-8688919-6136264', '112-1496519-6827432',
       '113-3245746-4521005', '111-3574642-5731411',
       '111-7454243-0089851', '112-2731660-4289835',
       '114-1596134-2427462', '114-3398304-3281857',
       '112-1539668-7355400', '112-7959155-8025009',
       '112-6692488-0454602', '113-8042807-6050610',
       '112-5223901-9475433', '113-3503305-5381856',
       '112-6139980-9065064', '111-0219710-1549069',
       '112-9835980-5942648', '113-0973671-6113869',
       '113-2905256-2248204', '112-9118722-4417024',
       '113-6596328-6833023', '112-8340669-7515444',
       '113-9878954-5329025', '113-4822864-1422626',
       '114-3348004-7024249', '113-5590810-6870650',
       '112-0562364-9713017', '114-1344293-9493859',
       '111-4097598-3304245', '113-6396592-049

In [37]:
import math
import numpy as np
cancel = pd.read_excel(r'E:/已取消.xlsx' )
cancel['平台SKU'] =np.nan
cancel = cancel[['销售交易号', '账号', '平台SKU','sku', '数量', '包裹编号','订单类型', '包裹状态', '创建时间']]
cancel= cancel.rename(columns={"销售交易号":"平台订单号","sku":"公司SKU","包裹编号":"包裹号","包裹状态":"订单状态","创建时间": "订单时间"})
cancel['平台SKU'] = cancel['平台SKU'].astype(str)
cancel

In [45]:
cancel= cancel.rename(columns={"销售交易号":"平台订单号","sku":"公司SKU","包裹编号":"包裹号","包裹状态":"订单状态","创建时间": "订单时间"})
cancel['平台SKU'] = cancel['平台SKU'].astype(str)
cancel

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单类型,订单状态,订单时间
0,114-5791259-5967441,信盒贸易,nan,USAN1029451-4,1,AK2022051306007,正常订单,已取消,2022-05-13 09:02:54
1,2871330482074,heman Walmart US,nan,USAN1034818-2,1,AK2022051303543,正常订单,已取消,2022-05-13 05:05:02
2,2871330482074,heman Walmart US,nan,USAN1034818-2,1,AK2022051303542,正常订单,已取消,2022-05-13 05:05:02
3,2871330187135,heman Walmart US,nan,USAN1022653-2,1,AK2022051302723,正常订单,已取消,2022-05-13 03:46:32
4,114-4037000-9684249,信盒贸易,nan,USAN1034691-5,1,AK2022051300375,正常订单,已取消,2022-05-13 00:40:32
...,...,...,...,...,...,...,...,...,...
32559,114-7797427-5557054,宫本,nan,USAN1015826-2,1,AK2020121700002,正常订单,已取消,2020-12-17 00:03:01
32560,111-9880623-6876257,宫本,nan,USAN1015826-2,1,AK2020121700001,正常订单,已取消,2020-12-17 00:02:05
32561,112-1273870-7245042,宫本,nan,USAN1011332-1,1,AK2020121600019,正常订单,已取消,2020-12-16 20:20:03
32562,113-4308694-3305032,宫本,nan,HOG101429-03,1,AK2020121600018,正常订单,已取消,2020-12-16 20:17:01


In [53]:
cancel['平台订单号'].unique()

array(['114-5791259-5967441', '2871330482074', '2871330187135', ...,
       '112-1273870-7245042', '113-4308694-3305032',
       '111-3554288-5334622'], dtype=object)

In [63]:
cancel[cancel['平台订单号'] == '112-6201342-7128260']

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单类型,订单状态,订单时间


In [66]:
import pandas as pd
merge = pd.merge(originzifa,cancel,on = ['平台订单号','账号','平台SKU','公司SKU','数量','包裹号','订单类型','订单状态','订单时间'],how = 'outer')
merge

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单类型,订单状态,订单时间
0,111-1984040-5269802,宫本,GB-103483-1,PTG103482-01,1,AK2020122500001,正常订单,已发货,2020-12-25 03:40:00
1,113-9678451-1602655,宫本,G-1015038-1,USAN1015038-1,1,AK2020122500005,正常订单,已发货,2020-12-25 05:57:00
2,112-8973470-8715413,宫本,USAN1011332-4,USAN1015826-2,1,AK2020122500013,正常订单,已发货,2020-12-25 08:57:00
3,111-2723955-5121007,宫本,G-15038-3,USAN1015038-3,1,AK2020122500006,正常订单,已发货,2020-12-25 05:57:00
4,113-5462561-8369054,宫本,GB-103483-2,PTG103482-02,1,AK2020122500012,正常订单,已发货,2020-12-25 08:57:00
...,...,...,...,...,...,...,...,...,...
1331625,114-7797427-5557054,宫本,nan,USAN1015826-2,1,AK2020121700002,正常订单,已取消,2020-12-17 00:03:01
1331626,111-9880623-6876257,宫本,nan,USAN1015826-2,1,AK2020121700001,正常订单,已取消,2020-12-17 00:02:05
1331627,112-1273870-7245042,宫本,nan,USAN1011332-1,1,AK2020121600019,正常订单,已取消,2020-12-16 20:20:03
1331628,113-4308694-3305032,宫本,nan,HOG101429-03,1,AK2020121600018,正常订单,已取消,2020-12-16 20:17:01


In [69]:
merge[merge["订单状态"].str.contains('已取消')&merge["订单类型"].str.contains('补寄')]

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单类型,订单状态,订单时间
19765,111-5127027-4278622,赫曼,L-USAN1017103-3+4,USAN1017103-4,1,NaN,补寄订单,已取消,2021-03-20 19:53:00
74985,112-3871065-0663469,赫曼,USAN1018799-5+6,USAN1018799-5,1,NaN,补寄订单,已取消,2021-03-21 11:25:00
120113,114-0834957-6478654,优瑞斯特,U-1015038-2,USAN1015038-2,1,NaN,补寄订单,已取消,2021-03-23 19:27:00
136910,NaN,品牌十一部-利芬组,NaN,NaN,1,NaN,补寄订单,已取消,2021-05-19 11:41:00
138351,111-2039913-0288202,赫曼,NaN,USAN1017103-2,1,NaN,补寄订单,已取消,2021-05-19 11:23:00
...,...,...,...,...,...,...,...,...,...
1331399,113-2392064-1874668,宫本,nan,USAN1015826-1,1,AK2021012400013,补寄订单,已取消,2021-01-24 02:59:00
1331402,114-5206626-0478645,宫本,nan,USAN1015826-1,1,AK2021011900096,补寄订单,已取消,2021-01-19 19:26:00
1331426,112-4670525-8085040,宫本,nan,USAN1019205,1,AK2021011000179,补寄订单,已取消,2021-01-10 23:39:00
1331427,112-4670525-8085040,宫本,nan,USAN1019208-2,1,AK2021011000178,补寄订单,已取消,2021-01-10 23:38:00


In [70]:
merge[merge['平台订单号'] == '112-4670525-8085040']

,平台订单号,账号,平台SKU,公司SKU,数量,包裹号,订单类型,订单状态,订单时间
98834,112-4670525-8085040,宫本,USAN1011332-3,USAN1015826-1,2,"AK2021010400146,AK2021010400147",正常订单,包裹处理中,2021-01-04 10:57:00
1331426,112-4670525-8085040,宫本,nan,USAN1019205,1,AK2021011000179,补寄订单,已取消,2021-01-10 23:39:00
1331427,112-4670525-8085040,宫本,nan,USAN1019208-2,1,AK2021011000178,补寄订单,已取消,2021-01-10 23:38:00
1331428,112-4670525-8085040,宫本,nan,USAN1019204,1,AK2021011000177,补寄订单,已取消,2021-01-10 23:38:00
